# Setup

In [6]:
%%capture
!pip install gradio openai==0.28
!pip install python-dotenv

In [5]:
import gradio as gr
import json
import openai
import warnings

warnings.filterwarnings("ignore")

# Import drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

# Load the .env file from Google Drive
import os
from dotenv import load_dotenv
load_dotenv(os.path.join('/content/drive/MyDrive', '.env'))

open_ai_key = os.getenv('OPEN_AI_KEY')
tmdbs_api_key = os.getenv('TMDB_API_KEY')
tmdbs_auth_token = os.getenv('TMDB_AUTH_TOKEN')

openai.api_key = open_ai_key

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Baseline

In [ ]:
# Function to process user input and selected categories
def process_input(prompt, music, books, tv_shows, movies):
    # Collect selected categories
    selected_categories = []
    if music:
        selected_categories.append("Music")
    if books:
        selected_categories.append("Books")
    if tv_shows:
        selected_categories.append("TV Shows")
    if movies:
        selected_categories.append("Movies")

    # Prepare the JSON response
    result = {
        "prompt": prompt,
        "categories": selected_categories
    }
    return result

# Function to call GPT API using the JSON output
def call_gpt_api(prompt_data):
    # Construct the instruction prompt for GPT
    instruction_prompt = (
        f"You are an entertainment recommendation assistant. "
        f"Based on the following user input and selected categories, generate recommendations:\n\n"
        f"Prompt: {prompt_data['prompt']}\n"
        f"Categories: {', '.join(prompt_data['categories'])}\n"
    )

    # Call OpenAI's GPT model
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # You can use "gpt-3.5-turbo" or "gpt-4"
            messages=[{"role": "user", "content": instruction_prompt}],
            max_tokens=200
        )
        # Extract the response text
        gpt_reply = response['choices'][0]['message']['content']
        return gpt_reply
    except Exception as e:
        return f"Error calling GPT API: {str(e)}"

# Define the interface elements
with gr.Blocks() as interface:
    gr.Markdown("### Entertainment Recommendation Input")
    prompt_textbox = gr.Textbox(label="Enter your prompt:", lines=4, placeholder="Type your recommendation query here...")
    with gr.Row():
        music = gr.Checkbox(label="Music")
        books = gr.Checkbox(label="Books")
        tv_shows = gr.Checkbox(label="TV Shows")
        movies = gr.Checkbox(label="Movies")
    gpt_output = gr.Textbox(label="GPT Recommendations", lines=6, interactive=False)
    submit_button = gr.Button("Submit")

    # Define interactions
    def process_and_generate(prompt, music, books, tv_shows, movies):
        # Generate JSON output internally
        prompt_data = process_input(prompt, music, books, tv_shows, movies)
        # Call GPT API with the generated JSON
        gpt_reply = call_gpt_api(prompt_data)
        return gpt_reply

    submit_button.click(
        process_and_generate,
        inputs=[prompt_textbox, music, books, tv_shows, movies],
        outputs=[gpt_output]
    )

# Launch the Gradio interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d2bbe4a560bb1be7f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Demo on a Movie and cross domain recommendations

In [ ]:
import requests
key = tmdbs_api_key
url = "https://api.themoviedb.org/3/authentication"

headers = {
    "accept": "application/json",
    "Authorization": tmdbs_auth_token
}

print("Let's check the access credentials to TheMovieDb...", end=' ')
response = requests.get(url, headers=headers)
print("Done")

print(response.text)


Let's check the access credentials to TheMovieDb... Done
{"success":true,"status_code":1,"status_message":"Success."}


In [ ]:
import os
import time

def download_data_by_movie_title(movie_name):
    all_data = []

    url = f'https://api.themoviedb.org/3/search/movie?query={movie_name}&api_key={key}&include_adult=false&language=en-US&page=1'
    response = requests.get(url, headers)

    if response.status_code == 200:
        all_data.append(response.json())
        print(f'Got: {movie_name}')
    else:
        print(f'Error: {response.status_code}')

    return all_data[0]['results'][0]

In [ ]:
movie = download_data_by_movie_title('Interstellar')
genres = movie['genre_ids']
overview = movie['overview']
title = movie['title']

Got: Interstellar


In [ ]:
import pandas as pd

url = "https://api.themoviedb.org/3/genre/movie/list?language=en"

response = requests.get(url, headers=headers)
genres_idx = pd.DataFrame(response.json()['genres'])

In [ ]:
genres_idx[genres_idx['id'] == 28]['name'].values[0]

'Action'

In [ ]:
genres = [genres_idx[genres_idx['id'] == genre]['name'].values[0] for genre in genres]
genres

['Adventure', 'Drama', 'Science Fiction']

In [ ]:
def get_movie_features(movie_name):
    movie = download_data_by_movie_title(movie_name)
    if not movie:
        return "Movie not found."

    genres = movie['genre_ids']
    overview = movie['overview']
    title = movie['title']
    genres = [genres_idx[genres_idx['id'] == genre]['name'].values[0] for genre in genres]

    return {'title': title, 'overview': overview, 'genres': genres}

In [ ]:
# Function to call GPT API
def generate_recommendations(movie_name, recommend_movies, recommend_music, recommend_tv_shows, recommend_books):
    # Get movie features
    movie_features = get_movie_features(movie_name)
    if isinstance(movie_features, str):  # If an error message is returned
        return movie_features

    # Construct GPT prompt
    gpt_prompt = (
        f"You are an entartainment recommendation assistant. Based on the following movie's features, "
        f"generate recommendations for the selected categories. Provide a chain of thought for each recommendation. "
    )

    # Add movie features to the prompt
    gpt_prompt += (
        f"\n\nMovie Title: {movie_features['title']}\n"
        f"Overview: {movie_features['overview']}\n"
        f"Genres: {', '.join(movie_features['genres'])}\n"
    )

    # Specify selected categories
    selected_categories = []
    if recommend_movies:
        selected_categories.append("Movies")
    if recommend_music:
        selected_categories.append("Music")
    if recommend_tv_shows:
        selected_categories.append("TV Shows")
    if recommend_books:
        selected_categories.append("Books")

    if not selected_categories:
        return "Please select at least one category for recommendations."

    gpt_prompt += f"Recommend content in the following categories: {', '.join(selected_categories)}."

    # Requesting GPT to explain the reasoning behind each recommendation
    gpt_prompt += "\n\nFor each recommendation, provide a brief explanation of why you are suggesting it."

    if recommend_music:
      gpt_prompt += "\n Recommend music content, with similar features to the user input. Give music content, such as singers, albums or specific songs."
    if recommend_tv_shows:
      gpt_prompt += "\n Recommend TV Shows, with similar features to the user input."
    if recommend_books:
      gpt_prompt += "\n Recommend books, with similar features to the user input."
    if recommend_movies:
      gpt_prompt += "\n Recommend movies, with similar features to the user input."

    # Call OpenAI's GPT API
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": gpt_prompt}],
            max_tokens=1024
        )
        # Extract the response text
        gpt_reply = response['choices'][0]['message']['content']
        return gpt_reply
    except Exception as e:
        return f"Error calling GPT API: {str(e)}"

# Gradio Interface
with gr.Blocks() as interface:
    gr.Markdown("### Entertainment Recommendation System")
    movie_input = gr.Textbox(label="Enter a movie you like:", placeholder="Type a movie name (e.g., The Hobbit)")
    with gr.Row():
        recommend_movies = gr.Checkbox(label="Movies")
        recommend_music = gr.Checkbox(label="Music")
        recommend_tv_shows = gr.Checkbox(label="TV Shows")
        recommend_books = gr.Checkbox(label="Books")
    #gpt_output = gr.Textbox(label="Recommendations", lines=6, interactive=False)
    gpt_output = gr.Markdown()
    submit_button = gr.Button("Submit", variant = "primary")

    # Define interaction
    def process_and_generate(movie_name, movies, music, tv_shows, books):
        return generate_recommendations(movie_name, movies, music, tv_shows, books)

    submit_button.click(
        process_and_generate,
        inputs=[movie_input, recommend_movies, recommend_music, recommend_tv_shows, recommend_books],
        outputs=[gpt_output]
    )

# Launch the Gradio interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ab1555bdbf07258ee7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
